In [1]:
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Appendix: Inspecting data

### 0) Importing all the data
* We import randomly the data into a training set, validation set and test set, ensuring that the same mouse models do not occur in the different sets.

In [2]:
# setting the data folder
data_folder = '../data'

In [3]:
# picking csv-files
csv_files = [f for f in os.listdir(data_folder) if f.endswith('.csv')]

# making a dictionary based on the genome letter in the mouse
group_dict = {}
for filename in csv_files:
    # splits out extension and get the genome name
    base_name = os.path.splitext(filename)[0]
    group_name = base_name[-5:]
    # storing the filename with the right genome
    group_dict.setdefault(group_name, []).append(filename)

In [4]:
# shuffling the data
group_names = list(group_dict.keys())
random.shuffle(group_names)

# creating train, val and test sets
num_groups = len(group_names)
train_size = int(0.6 * num_groups)
test_size = int(0.2 * num_groups)
val_size = num_groups - train_size - test_size

train_groups = group_names[:train_size]
test_groups = group_names[train_size:train_size + test_size]
val_groups = group_names[train_size + test_size:]

train_files = [filename for group in train_groups for filename in group_dict[group]]
test_files = [filename for group in test_groups for filename in group_dict[group]]
val_files = [filename for group in val_groups for filename in group_dict[group]]

In [5]:
# function that filters out the 'unknown' data

def load_and_filter(files_list):
    dfs = []
    for filename in files_list:
        file_path = os.path.join(data_folder, filename)
        df = pd.read_csv(file_path)
        # filters out unknown
        df = df[df['sleep_episode'] != 0] # unknown = 0
        # removes the time column
        df = df.drop(columns=['time'])
        dfs.append(df)
    # creates one long file with all the sleep scoring
    if dfs:
        return pd.concat(dfs, ignore_index=True)
    else:
        return pd.DataFrame()  # for empty dfs

train_df = load_and_filter(train_files)
test_df = load_and_filter(test_files)
val_df = load_and_filter(val_files)

### 1) Overview of the data

In [6]:
train_df.info

<bound method DataFrame.info of          delta_power  theta_power  sigma_power  beta_power  emg_power  \
0           0.035697     0.050050     0.024376    0.020170   0.012657   
1           0.038797     0.047070     0.020935    0.019491   0.031251   
2           0.030161     0.062763     0.023212    0.015587   0.018868   
3           0.041064     0.056119     0.023254    0.022701   0.036574   
4           0.055847     0.047321     0.026141    0.019780   0.029522   
...              ...          ...          ...         ...        ...   
1167096     0.054960     0.034170     0.013552    0.013920   0.097897   
1167097     0.069506     0.032111     0.021751    0.013156   0.092292   
1167098     0.047995     0.038328     0.016894    0.015349   0.092578   
1167099     0.054761     0.044430     0.021102    0.017827   0.092924   
1167100     0.074287     0.038952     0.017136    0.013118   0.094564   

         sleep_episode  
0                    1  
1                    1  
2               

### 2) Statistical summary

#### Theoretical background

- **Mean** ($\mu$): The average value of a dataset, calculated as:
  $$
  \mu = \frac{1}{N} \sum_{i=1}^{N} x_i
  $$

- **Standard Deviation** ($\sigma$): A measure of data dispersion around the mean:
  $$
  \sigma = \sqrt{\frac{1}{N} \sum_{i=1}^{N} (x_i - \mu)^2}
  $$

- **Variance** ($\sigma^2$): The square of the standard deviation:
  $$
  \sigma^2 = \frac{1}{N} \sum_{i=1}^{N} (x_i - \mu)^2
  $$

In [7]:
train_df.describe()

,delta_power,theta_power,sigma_power,beta_power,emg_power,sleep_episode
count,1.167101e+06,1.167101e+06,1.167101e+06,1.167101e+06,1.167101e+06,1.167101e+06
mean,4.911652e-02,4.547190e-02,2.515342e-02,2.127904e-02,1.647124e-01,1.538742e+00
std,7.558375e-02,2.922236e-02,1.645387e-02,1.113252e-02,2.390643e-01,7.772455e-01
min,-3.463106e-01,-1.914635e-01,-8.540071e-02,-4.784966e-02,-4.138887e-01,1.000000e+00
25%,2.653000e-02,3.064264e-02,1.568016e-02,1.448491e-02,5.256403e-02,1.000000e+00
50%,3.823673e-02,3.936916e-02,2.078475e-02,1.858644e-02,9.279487e-02,1.000000e+00
75%,5.688062e-02,5.316248e-02,2.915336e-02,2.505387e-02,1.604122e-01,2.000000e+00
max,3.277576e+00,1.592684e+00,7.065098e-01,4.789401e-01,5.767568e+00,4.000000e+00


#### Observations in the data set
**Count:** Each variable has 1,148,392 data points, indicating no missing values across the columns.\
**Means:** All the power variables have low mean values, suggesting the data is centered around small magnitudes. 

Note: The low value of `sleep_episode` mean (1.57) indicates that a majority of episodes belong to `Wake (class 1)`.

**Standard deviations:** `emg_power` is the power variable with the highest std. This suggests that the muscle waves have a broader set of values than the brain waves: they latter are more tightly centered around their mean.

### 3) Possible missing values

As we saw in 2) there seem to be no missing or NaN-values in the dataset.

In [ ]:
missing_values = train_df.isnull().sum()
print(missing_values)

### 4) Examining class balance and weighting

In imbalanced datasets, certain classes dominate the training process, leading to biased models. To mitigate this issue, we calculate weights for each class based on their frequencies. The weights are defined as inversely proportional to the frequency of each class, ensuring that underrepresented classes are given higher importance:

$$
w_c = \frac{1}{n_c}
$$

where $n_c$ is the number of samples in class $c$.

These weights are then normalized to sum to 1:

$$
\tilde{w}_c = \frac{w_c}{\sum_{c=1}^{C} w_c}
$$
This ensures that the underrepresented classes are given higher importance during training.


In [ ]:
class_counts = train_df['sleep_episode'].value_counts()
class_weights = 1/class_counts
normalized_weights = class_weights / class_weights.sum()
print("Class balance\n", class_counts)
print("\nClass weights (normalized)\n", normalized_weights)

### 5) Correlation Analysis for Further Intuition

A **correlation matrix** provides an overview of the linear relationships between features in a dataset. The values in the matrix range from $-1$ to $1$, where:

- Values close to $1$ indicate a strong positive relationship: as one feature increases, so does the other.
- Values close to $-1$ indicate a strong negative relationship: as one feature increases, the other decreases.
- Values close to $0$ indicate little to no linear relationship between the features.

Features that are highly correlated could provide redundant information to the model. By identifying these, we could simplify the dataset without losing valuable information, potentially improving model efficiency and interpretability.

[Reference](https://compphysics.github.io/MachineLearning/doc/LectureNotes/_build/html/chapter8.html#correlation-matrix)

**Pair Plots for Visual Inspection**

To complement the correlation matrix, pair plots can be used to visualize pairwise relationships between features. The pair plot creates scatter plot between every two variables in our dataset (source: https://seaborn.pydata.org/generated/seaborn.pairplot.html). If the data is highly correlated the plot will be more diagonal, and in the opposite case the data forms other "lump-like" structures.

In [ ]:
# setting up a directory to save the plots in
output_folder = 'plots'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [ ]:
# Set LaTeX font style and font size
plt.rcParams.update({
    "text.usetex": True,
    "font.size": 16,
    "font.family": "serif",
})

# Compute correlation matrix
corr_matrix = train_df.drop(columns=['sleep_episode']).corr()

# Update class labels
class_labels = [r'$\delta$', r'$\theta$', r'$\sigma$', r'$\beta$', 'EMG']
class_labels = ['Delta', 'Theta', 'Sigma', 'Beta', 'EMG']

# Create heatmap with updated labels
plt.figure(figsize=(6, 8))  # Adjust to keep the plot square
heatmap = sns.heatmap(
    corr_matrix,
    annot=True,
    cmap='coolwarm',
    cbar=None,
    xticklabels=class_labels,
    yticklabels=class_labels,
    square=True  # Ensures the cells are square
)

# Save and show plot
plt.savefig(os.path.join(output_folder, 'corr_matrix.png'), dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# making a test fraction of 10% of the entire training set
sample_fraction = 0.1
sampled_df = train_df.sample(frac=sample_fraction, random_state=42)

# plotting pairplot
pair_plot = sns.pairplot(sampled_df, hue='sleep_episode', markers=["o", "s", "D", "*"])
pair_plot.savefig(os.path.join(output_folder, 'pairplot.png'), dpi=300)
plt.show()

From this plot we can read that `theta_power` shares data with `sigma_power` and `beta_power`, and the same is true within `sigma_power` and `beta_power`. It *could* then be that beta and sigma are not good predictors.

### 6) Further examining feature importance with a Random Forest Classifier

We could gain further insight into the importance of each predictor by quickly training an off the shelf Random Forest network. 

Random Forests are ensemble learning methods that build multiple **decision trees** during training and aggregate their results to improve predictions. They are particularly useful for examining feature importance.

The impurity reduction, often measured using the Gini impurity or entropy, is calculated as:
$$
\text{Impurity Reduction} = \text{Impurity}_{\text{parent}} - \sum_{i} \frac{N_i}{N_{\text{parent}}} \cdot \text{Impurity}_{\text{child}, i}
$$
where $N_i$ is the number of samples in the child node and $N_{\text{parent}}$ is the number of samples in the parent node.

Feature importance is aggregated over all splits and all trees in the forest. Features that appear frequently at the top levels of trees or significantly reduce impurity are assigned higher importance.

[Reference](https://compphysics.github.io/MachineLearning/doc/LectureNotes/_build/html/week46.html).

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(sampled_df[['delta_power', 'theta_power', 'sigma_power', 'beta_power', 'emg_power']], sampled_df['sleep_episode'])
feature_importances = rf.feature_importances_
print(dict(zip(['delta_power', 'theta_power', 'sigma_power', 'beta_power', 'emg_power'], feature_importances)))

From this result, we can see that `delta_power` is the most significant predictor, followed by `emg_power`and `sigma_power`

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Velg numeriske kolonner som skal normaliseres
numerical_cols = ['delta_power', 'theta_power', 'sigma_power', 'beta_power', 'emg_power']

# Opprett og tren en MinMaxScaler på treningsdataene
scaler = MinMaxScaler()
train_df[numerical_cols] = scaler.fit_transform(train_df[numerical_cols])

# Bruk den trenede skaleren på validerings- og testdataene
val_df[numerical_cols] = scaler.transform(val_df[numerical_cols])
test_df[numerical_cols] = scaler.transform(test_df[numerical_cols])


# Feature engineering

### 7) Creating power band ratios

With the intuition gained in 5) and 6), we could now remove some of the "raw" powerbands, and add some ratios, namely `delta_power/theta_power`, `sigma_power/beta_power`, `theta_power/sigma_power` and `emg_power/delta_power` to the data set. Then we could do another correlation matrix, to see if this has improved the model's predictors.

Creating ratios between features can normalize the data and highlight relationships. For example:
$$
\text{delta\_theta\_ratio} = \frac{\text{delta\_power}}{\text{theta\_power}}
$$
Ratios can reduce scale dependencies and emphasize relative changes, which are often more informative for classification tasks.


In [ ]:
train_df['delta_theta_ratio'] = train_df['delta_power'] / train_df['theta_power']
val_df['delta_theta_ratio'] = val_df['delta_power'] / val_df['theta_power']
test_df['delta_theta_ratio'] = test_df['delta_power'] / test_df['theta_power']

train_df['sigma_beta_ratio'] = train_df['sigma_power'] / train_df['beta_power']
val_df['sigma_beta_ratio'] = val_df['sigma_power'] / val_df['beta_power']
test_df['sigma_beta_ratio'] = test_df['sigma_power'] / test_df['beta_power']

train_df['theta_sigma_ratio'] = train_df['theta_power'] / train_df['sigma_power']
val_df['theta_sigma_ratio'] = val_df['theta_power'] / val_df['sigma_power']
test_df['theta_sigma_ratio'] = test_df['theta_power'] / test_df['sigma_power']

train_df['emg_delta_ratio'] = train_df['emg_power'] / train_df['delta_power']
val_df['emg_delta_ratio'] = val_df['emg_power'] / val_df['delta_power']
test_df['emg_delta_ratio'] = test_df['emg_power'] / test_df['delta_power']

# moving 'sleep_episode' to the back
train_df = train_df[[col for col in train_df.columns if col != 'sleep_episode'] + ['sleep_episode']]
val_df = val_df[[col for col in val_df.columns if col != 'sleep_episode'] + ['sleep_episode']]
test_df = test_df[[col for col in test_df.columns if col != 'sleep_episode'] + ['sleep_episode']]

In [ ]:
train_df

In [ ]:
# removing "raw" variables
columns_to_remove = ['theta_power', 'beta_power']
#columns_to_remove = ['delta_power', 'theta_power', 'beta_power']

train_df = train_df.drop(columns=columns_to_remove)
test_df = test_df.drop(columns=columns_to_remove)
val_df = val_df.drop(columns=columns_to_remove)

In [ ]:
train_df

### 8) New plots after feature engineering

In [ ]:
sample_fraction = 0.1
sampled_df = train_df.sample(frac=sample_fraction, random_state=42)

# pair plot
pair_plot = sns.pairplot(sampled_df, hue='sleep_episode', markers=["o", "s", "D", "*"])
pair_plot.savefig(os.path.join(output_folder, 'pairplot_after_engineering.png'), dpi=300)
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Set LaTeX font style and font size
plt.rcParams.update({
    "text.usetex": True,
    "font.size": 16,
    "font.family": "serif",
})

# Compute correlation matrix
corr_matrix = train_df.drop(columns=['sleep_episode']).corr()

# Update class labels
class_labels = [r'$\delta$', r'$\theta$', r'$\sigma$', r'$\beta$', 'EMG']
class_labels = ['Delta', 'Sigma', 'EMG', r'$\delta/\theta$', r'$\sigma/\beta$', r'$\theta/\sigma$', r'EMG$/\delta$']

# Create heatmap with updated labels
plt.figure(figsize=(6, 8))  # Adjust to keep the plot square
heatmap = sns.heatmap(
    corr_matrix,
    annot=True,
    cmap='coolwarm',
    cbar=None,
    xticklabels=class_labels,
    yticklabels=class_labels,
    square=True  # Ensures the cells are square
)

# Save and show plot
plt.savefig(os.path.join(output_folder, 'corr_matrix_after_engineering.png'), dpi=300, bbox_inches='tight')
plt.show()


### Future Investigations

- Train models using only the most predictive raw features ($\text{delta\_power}, \text{sigma\_power}, \text{emg\_power}$).
- Add power ratios (e.g., $\frac{\text{delta\_power}}{\text{theta\_power}}$) and evaluate their impact.
- Use **Principal Component Analysis (PCA)** to identify the most critical feature combinations, further reducing redundancy.